In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
import numpy as np
from datetime import datetime

In [ ]:
vfacts_df = pd.read_csv('vfacts_sales_df.csv', index_col=0)
abs_df = pd.read_csv('abs_df.csv', index_col=0)
evc_df = pd.read_csv('evc_df.csv', index_col=0)
nsw_df = pd.read_csv('nsw_rego_data.csv', index_col=0)

Compare the national aggregation of ABS stock over time with the VFACTS sales over time (both in per month values)

In [ ]:
# Extract the year in decimal form from the date to use as x-axis for plotting and fitting
def get_year_from_date(col):
    return [
    datetime.strptime(date, "%Y-%m").year + datetime.strptime(date, "%Y-%m").month / 12
    for date in col
]

vfacts_year = get_year_from_date(vfacts_df.columns[3:])
evc_year = get_year_from_date(evc_df['Year'])

Define the two and three parameter adoption curves

In [ ]:
def adoption(t, a, b):
    return 1 / (1 + np.exp(-a * ((t-2020) - b)))

def adoption3(t, a, b, c):
   return c / (1 + np.exp(-a * ((t-2020) - b))) 

In [ ]:
def get_calibrated_adoption_curve(tin, f, tout, calibration_points = [(2040,0.99,1)]):

    alpha0 = 0.5
    beta0 = 8
    gamma0 = 1.5

    p_guess = [alpha0, beta0]
    # p_guess3 = [alpha0, beta0, gamma0]

    t1 = tin.copy()
    f1 = f.copy()
    sigma = [1 for i in range(len(t1))]

    sigma[-1] = 0.1

    for p in calibration_points:
        t1.append(p[0])
        f1.append(p[1])
        if len(p) == 3:
            sigma.append(p[2])
        else:
            sigma.append(1)

    p2_opt = curve_fit(f=adoption, xdata=t1, ydata=f1, p0=p_guess, sigma=sigma)
    alpha2 = p2_opt[0][0]
    beta2 = p2_opt[0][1]

    p_guess3 = [alpha2, beta2, 1]
    p3_opt = curve_fit(f=adoption3, xdata=t1, ydata=f1, p0=p_guess3, sigma=sigma)
    alpha3 = p3_opt[0][0]
    beta3 = p3_opt[0][1]
    gamma3 = p3_opt[0][2]

    return adoption(tout, alpha2, beta2), adoption3(tout, alpha3, beta3, gamma3)


Visualise the data and fit

In [ ]:
nsw_df

,Date,Light Passenger Vehicles Battery Electric,Light Goods Vehicles Battery Electric,Light Goods Vehicles All,Light Passenger Vehicles All,Total Vehicles,Percent EV Passenger Vehicles,Percent EV Goods Vehicles,NSW Percent EV (Total),Year
0,2022-07-31,186,0,6491,18315,24806,1.015561,0.000000,0.749819,2022.580822
1,2022-08-31,1785,0,6971,21250,28221,8.400000,0.000000,6.325077,2022.665753
2,2022-09-30,2353,7,7607,20107,27714,11.702392,0.092021,8.515552,2022.747945
3,2022-10-31,980,0,6994,18328,25322,5.347010,0.000000,3.870152,2022.832877
4,2022-11-30,1246,24,6887,20998,27885,5.933898,0.348483,4.554420,2022.915068
5,2022-12-31,970,32,6710,17759,24469,5.462019,0.476900,4.094977,2023.000000
6,2023-01-31,1955,12,6216,19373,25589,10.091364,0.193050,7.686897,2023.084932
7,2023-02-28,1778,34,4931,14028,18959,12.674651,0.689515,9.557466,2023.161644
8,2023-03-31,300,0,865,2827,3692,10.611956,0.000000,8.125677,2023.246575
9,2023-04-30,433,0,1662,6442,8104,6.721515,0.000000,5.343040,2023.328767


In [ ]:
[r for r in range(3)]

[0, 1, 2]

In [ ]:
y = 2022
l = nsw_df["Year"].to_list()
tmp = [i for i in range(len(l)) if l[i] > y and l[i] <= y + 1]
nsw_df["Light Passenger Vehicles Battery Electric"].iloc[tmp].sum()

7520

just a note: Vehicle stock, scrapping rate ABS Catalogue No. 9309.0 - Motor Vehicle Census, Australia, 31 Jan 2021 (ABS, 2021a) 

In [ ]:
def nsw_stock_rate_plot():

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=nsw_df["Year"].to_list(),
            y=(nsw_df["Light Passenger Vehicles Battery Electric"]*12).to_list(),
            mode="lines",
            name="NSW New Registrations",
        )
    )

    year = []
    value = []
    for y in range(2020, 2030):
        tmp = [i for i in nsw_df["Year"].to_list() if i > y and i <= y + 1]
        if len(tmp) > 0:
            year.append(y)
            value.append(sum(nsw_df["Light Passenger Vehicles Battery Electric"].to_list()[tmp]))

    fig.add_trace(
        go.Scatter(
            x=vfacts_year,
            y=vfacts_df[vfacts_df["State"]=="NSW"].groupby("Fuel Type").sum().loc["Electric"].values.tolist(),
            mode="lines",
            name="VFACTS NSW Electric Sales",))

    fig.update_layout(title_text="NSW EV Stock Change Per Year")

    fig.show()

nsw_stock_rate_plot()

TypeError: list indices must be integers or slices, not list

In [ ]:
def national_plot(fuel_type):
    # ABS stock data
    total_stock_change_per_year = (
        abs_df.astype({"Postcode": "string"}).groupby(["Fuel Type"]).sum().diff(axis=1).sum(axis=0)
    )
    fueltype_stock_change_per_year = (
        abs_df.astype({"Postcode": "string"}).groupby(["Fuel Type"]).sum().loc[fuel_type].diff()
    )  # change to vehicle stock per year
    fueltype_stock_change_per_year_percent = (
        fueltype_stock_change_per_year / total_stock_change_per_year * 100
    )

    # VFACTS sales data
    total_sales_per_year = vfacts_df.groupby(["Fuel Type"]).sum().sum(axis=0) * 12
    fueltype_sales_per_year = vfacts_df.groupby(["Fuel Type"]).sum().loc[fuel_type] * 12
    fueltype_sales_per_year_percent = (
        fueltype_sales_per_year / total_sales_per_year * 100
    )

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    x1 = fueltype_stock_change_per_year.index.astype(int).to_list()
    y1 = fueltype_stock_change_per_year.values.tolist()

    # Add traces
    fig.add_trace(
        go.Scatter(x=x1, y=y1, name="ABS Stock Change", opacity=0.5, line={'width': 5}),
        secondary_y=False,
    )

    x2 = vfacts_year
    y2 = fueltype_sales_per_year.values.tolist()

    fig.add_trace(
        go.Scatter(x=x2, y=y2, name="VFACTS Sales", opacity=0.5, line={'width': 5}),
        secondary_y=False,
    )

    x3 = fueltype_stock_change_per_year.index.astype(int).to_list()
    y3 = fueltype_stock_change_per_year_percent.values.tolist()

    fig.add_trace(
        go.Scatter(x=x3, y=y3, name="ABS Stock Change Percent"),
        secondary_y=True,
    )

    x4 = vfacts_year
    y4 = fueltype_sales_per_year_percent.values.tolist()

    fig.add_trace(
        go.Scatter(x=x4, y=y4, name="VFACTS Sales Percent"),
        secondary_y=True,
    )

    three_par = False

    x5 = np.linspace(2020, 2040)
    calibration_points = [(2026, 0.8), (2040, 0.99)]
    y5, y6 = get_calibrated_adoption_curve(
        x4, [_y / 100 for _y in y4], x5, calibration_points=calibration_points
    )
    y5 = [_y * 100 for _y in y5]
    y6 = [_y * 100 for _y in y6]

    yfit = y5
    if three_par:
        yfit = y6

    fig.add_trace(
        go.Scatter(
        x=[f[0] for f in calibration_points],
        y=[f[1] * 100 for f in calibration_points],
        name="Calibration Points",
        marker=dict(size=8, color="blue", symbol='circle-open'),
        mode="markers"),secondary_y=True
    )

    fig.add_trace(
        go.Scatter(x=x5, y=yfit, name="Adoption Curve (VFACTS)", line={"dash": "dash"}),
        secondary_y=True,
    )

    x7 = evc_year
    y7 = evc_df["BEV Sales Percent"].tolist()
    fig.add_trace(
        go.Scatter(x=x7, y=y7, name="EVC Sales Percent"),
        secondary_y=True,
    )

    yfit2, yfit3 = get_calibrated_adoption_curve(
        x7, [_y / 100 for _y in y7], x5, calibration_points=calibration_points
    )
    yfit2 = [_y * 100 for _y in yfit2]
    yfit3 = [_y * 100 for _y in yfit3]

    yfit = yfit2
    if three_par:
        yfit = yfit3

    fig.add_trace(
        go.Scatter(x=x5, y=yfit, name="Adoption Curve (EVC)", line={"dash": "dash"}),
        secondary_y=True,
    )

    x8 = nsw_df["Year"].tolist()
    y8 = nsw_df["NSW Percent EV (Total)"].tolist()
    fig.add_trace(
        go.Scatter(x=x8, y=y8, name="NSW Percent EV (Total)"),
        secondary_y=True,
    )

    yfit2, yfit3 = get_calibrated_adoption_curve(
        x8, [_y / 100 for _y in y8], x5, calibration_points=calibration_points
    )
    yfit2 = [_y * 100 for _y in yfit2]
    yfit3 = [_y * 100 for _y in yfit3]

    yfit = yfit2
    if three_par:
        yfit = yfit3

    fig.add_trace(
        go.Scatter(x=x5, y=yfit, name="Adoption Curve (NSW)", line={"dash": "dash"}),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="Sales & Stock Change Data for Fuel Type = " + fuel_type
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Year")

    # Set y-axes titles
    fig.update_yaxes(title_text="Vehicles Per Year", secondary_y=False)
    fig.update_yaxes(title_text="Percent of All Fuel Types", secondary_y=True)

    fig.show()


national_plot("Electric")


In [ ]:
# select the state and vehicle type variables from those available in State and Vehicle Type columns

state = ''
vehicle_type = 'Passenger'

if state in vfacts_df['State'].unique():
    df = vfacts_df.loc[(vfacts_df['State'] == state)]
    df = df.drop(columns=['State'])
else:
    df = vfacts_df.groupby(['Vehicle Type', 'Fuel Type']).sum()
    df.reset_index(inplace=True)

if vehicle_type in df['Vehicle Type'].unique():
    df = df.loc[df['Vehicle Type'] == vehicle_type]
    df = df.drop(columns=['Vehicle Type'])
else:
    df = df.groupby(['Fuel Type']).sum()
    df.reset_index(inplace=True)

df = df.melt(
    id_vars=['Fuel Type'],
    var_name='Date',
    value_name='Sales Per Month',
)
total_sales = df.groupby(['Date']).sum()
percent_sales = [
    r['Sales Per Month'] / total_sales.loc[r['Date']]['Sales Per Month'] * 100
    for i, r in df.iterrows()
]
df['Percent Sales Per Month'] = percent_sales
df


,Fuel Type,Date,Sales Per Month,Percent Sales Per Month
0,Diesel,2020-09,5580.000000,2.557740
1,Electric,2020-09,934.666667,0.428429
2,Hybrid,2020-09,23548.000000,10.793847
3,Hydrogen,2020-09,0.000000,0.000000
4,PHEV,2020-09,345.333333,0.158293
5,Petrol,2020-09,187753.333333,86.061691
6,Diesel,2020-12,5282.000000,2.378176
7,Electric,2020-12,939.000000,0.422777
8,Hybrid,2020-12,25484.000000,11.473956
9,Hydrogen,2020-12,0.000000,0.000000


In [ ]:
abs_df

,State,Postcode,Vehicle Type,Fuel Type,2013,2014,2015,2016,2017,2018,2019,2020,2021
6,NT,800,Light Commercial,Diesel,1303.0,1384.0,1413.0,1428.0,1633.0,1615.0,1565.0,1348.0,1478.0
7,NT,800,Passenger,Diesel,957.0,1037.0,936.0,1010.0,1148.0,1289.0,1312.0,1109.0,1021.0
8,NT,801,Light Commercial,Diesel,40.0,46.0,39.0,40.0,45.0,56.0,74.0,68.0,6.0
9,NT,801,Passenger,Diesel,17.0,33.0,31.0,36.0,32.0,33.0,42.0,29.0,0.0
10,NT,802,Light Commercial,Diesel,0.0,0.0,3.0,5.0,3.0,4.0,6.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41515,WA,6182,Passenger,LPG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
41516,WA,6182,Light Commercial,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
41517,WA,6182,Passenger,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
41518,WA,6182,Light Commercial,Petrol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
national_passenger_df = df

In [ ]:
national_df = df

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

def plot_sales(df):
    subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig1 = px.area(df, x='Date', y='Sales Per Month', color='Fuel Type')
    for i in range(len(fig1['data'])):
        fig1['data'][i]['line']['width']=0
    fig2 = px.line(df, x='Date', y='Percent Sales Per Month', color='Fuel Type')
    fig2.update_traces(yaxis='y2')
    subplot_fig.add_traces(fig1.data + fig2.data)
    subplot_fig.show()

plot_sales(national_df)
plot_sales(national_passenger_df)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=75de7644-8bd4-4ecc-bdb2-2c9ef0ed94e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>